#### Load GCP Environment

In [129]:
import os
import pandas as pd
import numpy as np
import datetime
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/michaelrazumovsky/Documents/Personal Studying/Machine Learning/BigQuery/Census Project/pandas-project-270620-076d933a32ff.json'
pd.options.display.max_columns = 500
from google.cloud import bigquery
bigquery_client= bigquery.Client()

#### Pull Census County Data From 2010-2016 and Join With FIPS Data

In [130]:
def bigquery_pull():
    df_acs = bigquery_client.query("""Select * from bigquery-public-data.sdoh_bea_cainc30.fips where Year != "2018-01-01" """).to_dataframe()
    df_acs['Year'] = pd.to_datetime(df_acs['Year'])
    df_dict_train = {}
    df_dict_test = {}
    df_merge_train = {}
    train_merge_list = []
    global df_train, df_test
    
    for i in range(2012,2017):
        df_dict_train[f'df_{i}'] = bigquery_client.query(f"""select * from bigquery-public-data.census_bureau_acs.county_{i}_5yr""").to_dataframe()
        df_dict_train[f'df_{i}']['year'] = pd.to_datetime(f'{i}-01-01')
        df_merge_train[f'df_{i}'] = pd.merge(df_dict_train[f'df_{i}'],df_acs,how='left',left_on=['geo_id','year'],right_on = ['GeoFIPS',"Year"])
    for i in df_merge_train:
        train_merge_list.append(df_merge_train[i])
    df_train = pd.concat(train_merge_list,axis=0)

    
    df_dict_test['df_2017'] = bigquery_client.query("""select * from bigquery-public-data.census_bureau_acs.county_2017_5yr""").to_dataframe()
    df_dict_test['df_2017']['year'] = pd.to_datetime('2017-01-01')
    df_test = pd.merge(df_dict_test['df_2017'],df_acs,how='left',left_on=['geo_id','year'],right_on = ['GeoFIPS',"Year"])         
    
    return df_train, df_test
train_pull, test_pull = bigquery_pull()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [136]:
def clean_train(train):
    train.dropna(subset=['Year','GeoFIPS'],inplace=True)
    for i in train:
        if train[i].isna().sum()>0:
            train[i].fillna(value=train[i].median(),inplace=True)
    return train

def clean_test(test,train):
    test.dropna(subset=['Year','GeoFIPS'],inplace=True)
    for i in test:
        if test[i].isna().sum()>3000:
            test[i].fillna(value=train[i].median(),inplace=True)
        if (test[i].isna().sum()>0) & (test[i].isna().sum()<=3000):
            test[i].fillna(value=test[i].median(),inplace=True)
    return test
df_train = clean_train(train_pull)
df_test = clean_test(test_pull,train_pull)

In [138]:
df_test.isna().sum()


geo_id                                           0
nonfamily_households                             0
family_households                                0
median_year_structure_built                      0
rent_burden_not_computed                         0
rent_over_50_percent                             0
rent_40_to_50_percent                            0
rent_35_to_40_percent                            0
rent_30_to_35_percent                            0
rent_25_to_30_percent                            0
rent_20_to_25_percent                            0
rent_15_to_20_percent                            0
rent_10_to_15_percent                            0
rent_under_10_percent                            0
total_pop                                        0
male_pop                                         0
female_pop                                       0
median_age                                       0
white_pop                                        0
black_pop                      

In [83]:
df_train.isna().sum()[df_train.isna().sum()>0].keys()

Index(['Dividends_interest_rent', 'Earnings_by_place_of_work',
       'Earnings_per_job_avg', 'Employer_contrib_govt_and_social_insurance',
       'Employer_contrib_pension_and_insurance', 'Farm_proprietors_employment',
       'Farm_proprietors_income', 'GeoFIPS', 'GeoName',
       'Income_maintenance_benefits', 'Net_earnings_by_place_of_residence',
       'Nonfarm_proprietors_employment', 'Nonfarm_proprietors_income',
       'Nonfarm_proprietors_income_avg', 'Percapita_dividends_interest_rent',
       'Percapita_income_maintenance_benefits', 'Percapita_net_earnings',
       'Percapita_personal_current_transfer_receipts',
       'Percapita_personal_income', 'Percapita_retirement_and_other',
       'Percapita_unemployment_insurance_compensation',
       'Personal_current_transfer_receipts', 'Personal_income', 'Population',
       'Proprietors_employment', 'Proprietors_income', 'Retirement_and_other',
       'Total_employment', 'Unemployment_insurance',
       'Wage_and_salary_employment

In [84]:
df_train.isna().sum()[df_train.isna().sum()>0]

Dividends_interest_rent                                         663
Earnings_by_place_of_work                                       663
Earnings_per_job_avg                                            663
Employer_contrib_govt_and_social_insurance                      663
Employer_contrib_pension_and_insurance                          663
Farm_proprietors_employment                                     663
Farm_proprietors_income                                         663
GeoFIPS                                                         663
GeoName                                                         663
Income_maintenance_benefits                                     663
Net_earnings_by_place_of_residence                              663
Nonfarm_proprietors_employment                                  663
Nonfarm_proprietors_income                                      663
Nonfarm_proprietors_income_avg                                  663
Percapita_dividends_interest_rent               

In [106]:
xx = df_train.dropna(subset=['Year','GeoFIPS'])
xx.isna().sum()[df_train.dropna(subset=['Year','GeoFIPS']).isna().sum()>0]
#xx[xx['pop_15_and_over'].isna()]
#xx.columns.get_loc('pop_15_and_over')
#xx['pop_15_and_over'].quantile(q=[0.25,.5,.75,1])
for i in xx:
    if xx[i].isna().sum()>0:
        xx[i].fillna(value=xx[i].median(),inplace=True)

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [111]:
xx.isna().sum()
len(xx)

15439

In [98]:
x = []
for i in df_train.isna().sum()[df_train.isna().sum()>=3880].keys():
    x.append(df_train.columns.get_loc(i))
x
df_train.iloc[:,x]

,pop_15_and_over,pop_divorced,pop_never_married,pop_now_married,pop_separated,pop_widowed
0,3597,235,614,2241,56,425
1,1198,124,170,788,8,106
2,2755,311,453,1613,39,336
3,1606,65,290,1054,28,168
4,2943,184,595,1809,41,314
5,1134,58,312,690,8,66
6,7372,1184,1585,3835,145,616
7,3909,323,625,2549,40,321
8,7491,1126,3317,2321,177,534
9,3354,494,501,2094,27,224


In [77]:
df_train[df_train['Year'].isna()&df_train['GeoFIPS'].isna()]

,Dividends_interest_rent,Earnings_by_place_of_work,Earnings_per_job_avg,Employer_contrib_govt_and_social_insurance,Employer_contrib_pension_and_insurance,Farm_proprietors_employment,Farm_proprietors_income,GeoFIPS,GeoName,Income_maintenance_benefits,Net_earnings_by_place_of_residence,Nonfarm_proprietors_employment,Nonfarm_proprietors_income,Nonfarm_proprietors_income_avg,Percapita_dividends_interest_rent,Percapita_income_maintenance_benefits,Percapita_net_earnings,Percapita_personal_current_transfer_receipts,Percapita_personal_income,Percapita_retirement_and_other,Percapita_unemployment_insurance_compensation,Personal_current_transfer_receipts,Personal_income,Population,Proprietors_employment,Proprietors_income,Retirement_and_other,Total_employment,Unemployment_insurance,Wage_and_salary_employment,Wages_and_salaries,Wages_and_salaries_avg,Wages_and_salaries_supplement,Year,aggregate_travel_time_to_work,amerindian_including_hispanic,amerindian_pop,armed_forces,asian_including_hispanic,asian_male_45_54,asian_male_55_64,asian_pop,associates_degree,bachelors_degree,bachelors_degree_2,bachelors_degree_or_higher_25_64,black_including_hispanic,black_male_45_54,black_male_55_64,black_pop,children,children_in_single_female_hh,civilian_labor_force,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_39_mins,commute_35_44_mins,commute_40_44_mins,commute_45_59_mins,commute_5_9_mins,commute_60_89_mins,commute_60_more_mins,commute_90_more_mins,commute_less_10_mins,commuters_16_over,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_public_transportation,commuters_by_subway_or_elevated,commuters_drove_alone,different_house_year_ago_different_city,different_house_year_ago_same_city,do_date,dwellings_10_to_19_units,dwellings_1_units_attached,dwellings_1_units_detached,dwellings_20_to_49_units,dwellings_2_units,dwellings_3_to_4_units,dwellings_50_or_more_units,dwellings_5_to_9_units,employed_agriculture_forestry_fishing_hunting_mining,employed_arts_entertainment_recreation_accommodation_food,employed_construction,employed_education_health_social,employed_finance_insurance_real_estate,employed_information,employed_manufacturing,employed_other_services_not_public_admin,employed_pop,employed_public_administration,employed_retail_trade,employed_science_management_admin_waste,employed_transportation_warehousing_utilities,employed_wholesale_trade,families_with_young_children,family_households,father_in_labor_force_one_parent_families_with_young_children,father_one_parent_families_with_young_children,female_10_to_14,female_15_to_17,female_18_to_19,female_20,female_21,female_22_to_24,female_25_to_29,female_30_to_34,female_35_to_39,female_40_to_44,female_45_to_49,female_50_to_54,female_55_to_59,female_5_to_9,female_60_to_61,female_62_to_64,female_65_to_66,female_67_to_69,female_70_to_74,female_75_to_79,female_80_to_84,female_85_and_over,female_female_households,female_pop,female_under_5,four_more_cars,geo_id,gini_index,graduate_professional_degree,group_quarters,high_school_diploma,high_school_including_ged,hispanic_any_race,hispanic_male_45_54,hispanic_male_55_64,hispanic_pop,households,households_public_asst_or_food_stamps,households_retirement_income,housing_built_1939_or_earlier,housing_built_2000_to_2004,housing_built_2005_or_later,housing_units,housing_units_renter_occupied,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,income_100000_124999,income_10000_14999,income_125000_149999,income_150000_199999,income_15000_19999,income_200000_or_more,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,income_75000_99999,income_less_10000,income_per_capita,less_one_year_college,less_than_high_school_graduate,male_10_to_14,male_15_to_17,male_18_to_19,male_20,male_21,male_22_to_24,male_25_to_29,male_30_to_34,male_35_to_39,male_40_to_44,male_45_64_a

In [73]:
df_acs = bigquery_client.query("""Select * from bigquery-public-data.sdoh_bea_cainc30.fips where Year != "2018-01-01" AND Year != "2018-01-01" """).to_dataframe()
pd.to_datetime(df_acs['Year']).value_counts()

2012-01-01    3198
2017-01-01    3198
2016-01-01    3198
2015-01-01    3198
2014-01-01    3198
2013-01-01    3198
Name: Year, dtype: int64

In [78]:
pd.to_datetime(df_acs['Year']).value_counts()
df_acs['GeoFIPS'].isna().sum()

0

In [52]:
df_acs = bigquery_client.query("""Select * from bigquery-public-data.sdoh_bea_cainc30.fips""").to_dataframe()
#df_acs['Year'] = pd.to_datetime(df_acs['Year'])
df_acs.isna().sum()

Year                                               0
GeoFIPS                                            0
GeoName                                            0
Employer_contrib_pension_and_insurance           175
Employer_contrib_govt_and_social_insurance       175
Farm_proprietors_income                          175
Nonfarm_proprietors_income                       175
Farm_proprietors_employment                      175
Income_maintenance_benefits                      175
Nonfarm_proprietors_employment                   175
Percapita_income_maintenance_benefits            175
Percapita_retirement_and_other                   175
Percapita_unemployment_insurance_compensation    175
Proprietors_income                               175
Retirement_and_other                             175
Wages_and_salaries_supplement                    175
Unemployment_insurance                           175
Wages_and_salaries                               175
Nonfarm_proprietors_income_avg                

In [50]:
query_2010 = """select * from bigquery-public-data.census_bureau_acs.county_2010_5yr"""
query_2010_job = bigquery_client.query(query_2010)
df_10 = query_2010_job.to_dataframe()
df_10['year'] = pd.to_datetime('2010-01-01')
df_2010 = pd.merge(df_10,df_acs,how='left',left_on=['geo_id','year'],right_on = ['GeoFIPS',"Year"])

query_2011 = """select * from bigquery-public-data.census_bureau_acs.county_2011_5yr"""
query_2011_job = bigquery_client.query(query_2011)
df_11 = query_2011_job.to_dataframe()
df_11['year'] = pd.to_datetime('2011-01-01')
df_2011 = pd.merge(df_11,df_acs,how='left',left_on=['geo_id','year'],right_on = ['GeoFIPS',"Year"])

query_2012 = """select * from bigquery-public-data.census_bureau_acs.county_2012_5yr"""
query_2012_job = bigquery_client.query(query_2012)
df_12 = query_2012_job.to_dataframe()
df_12['year'] = pd.to_datetime('2012-01-01')
df_2012 = pd.merge(df_12,df_acs,how='left',left_on=['geo_id','year'],right_on = ['GeoFIPS',"Year"])

query_2013 = """select * from bigquery-public-data.census_bureau_acs.county_2013_5yr"""
query_2013_job = bigquery_client.query(query_2013)
df_13 = query_2013_job.to_dataframe()
df_13['year'] = pd.to_datetime('2013-01-01')
df_2013 = pd.merge(df_13,df_acs,how='left',left_on=['geo_id','year'],right_on = ['GeoFIPS',"Year"])

query_2014 = """select * from bigquery-public-data.census_bureau_acs.county_2014_5yr"""
query_2014_job = bigquery_client.query(query_2014)
df_14 = query_2014_job.to_dataframe()
df_14['year'] = pd.to_datetime('2014-01-01')
df_2014 = pd.merge(df_14,df_acs,how='left',left_on=['geo_id','year'],right_on = ['GeoFIPS',"Year"])

query_2015 = """select * from bigquery-public-data.census_bureau_acs.county_2015_5yr"""
query_2015_job = bigquery_client.query(query_2015)
df_15 = query_2015_job.to_dataframe()
df_15['year'] = pd.to_datetime('2015-01-01')
df_2015 = pd.merge(df_15,df_acs,how='left',left_on=['geo_id','year'],right_on = ['GeoFIPS',"Year"])

query_2016 = """select * from bigquery-public-data.census_bureau_acs.county_2016_5yr"""
query_2016_job = bigquery_client.query(query_2016)
df_16 = query_2016_job.to_dataframe()
df_16['year'] = pd.to_datetime('2016-01-01')
df_2016 = pd.merge(df_16,df_acs,how='left',left_on=['geo_id','year'],right_on = ['GeoFIPS',"Year"])

NameError: name 'df_acs' is not defined

In [ ]:
pd.concat([df_2010,df_2011,df_2012,df_2013,df_2014,df_2015,df_2016],axis=0)

In [53]:
query_2011_2014 = """with full_acs as (
select * from bigquery-public-data.census_bureau_acs.county_2011_5yr 
union all
select * from bigquery-public-data.census_bureau_acs.county_2012_5yr
union all
select * from bigquery-public-data.census_bureau_acs.county_2013_5yr
union all
select * from bigquery-public-data.census_bureau_acs.county_2014_5yr)


Select * from full_acs a
left join bigquery-public-data.sdoh_bea_cainc30.fips b
on b.GeoFIPS=a.geo_id;"""
query_2011_2014_job = bigquery_client.query(query_2011_2014)
df_2011_2014 = query_2011_2014_job.to_dataframe()

query_2010 = """with full_acs as (
select * from bigquery-public-data.census_bureau_acs.county_2010_5yr)

Select * from full_acs a
left join bigquery-public-data.sdoh_bea_cainc30.fips b
on b.GeoFIPS=a.geo_id;"""
query_2010_job = bigquery_client.query(query_2010)
df_2010 = query_2010_job.to_dataframe()

query_2015 = """with full_acs as (
select * from bigquery-public-data.census_bureau_acs.county_2015_5yr)

Select * from full_acs a
left join bigquery-public-data.sdoh_bea_cainc30.fips b
on b.GeoFIPS=a.geo_id;"""
query_2015_job = bigquery_client.query(query_2015)
df_2015 = query_2015_job.to_dataframe()

query_2016 = """with full_acs as (
select * from bigquery-public-data.census_bureau_acs.county_2016_5yr)

Select * from full_acs a
left join bigquery-public-data.sdoh_bea_cainc30.fips b
on b.GeoFIPS=a.geo_id;"""
query_2016_job = bigquery_client.query(query_2016)
df_2016 = query_2016_job.to_dataframe()

#### Create Train and Test Base Dataframes

In [ ]:
train_base = pd.concat([df_2010,df_2011_2014,df_2015,df_2016],axis=0)

query_2017 = """with full_acs as (
select * from bigquery-public-data.census_bureau_acs.county_2016_5yr)
Select * from full_acs a
left join bigquery-public-data.sdoh_bea_cainc30.fips b
on b.GeoFIPS=a.geo_id;"""
query_2017_job = bigquery_client.query(query_2017)
test_base = query_2017_job.to_dataframe()

In [ ]:
train_base[train_base.iloc[:,34:].duplicated()==False]

In [ ]:
df2[df2['associates_degree'].isna()].isna().sum()

In [ ]:
print(df_2010.shape)
print(df_2015.shape)
print(df_2016.shape)
df2.shape

In [ ]:
df_2010_acs = df_2010.iloc[:,:252]
df_2010[df_2010_acs.duplicated()==False]

In [ ]:
df_2010[df_2010['geo_id'] == "51510"]

In [ ]:
x = pd.DataFrame(data = {'a':[1,2,3,4],'b':[2,3,4,5]})
y = pd.DataFrame(data = {'b':[1,2,3,4],'c':[2,3,4,5]})
pd.concat([x,y])

In [ ]:
train_base = pd.concat([df_2010,df2,df_2015,df_2016],axis=0)

query_2017 = """with full_acs as (
select * from bigquery-public-data.census_bureau_acs.county_2016_5yr)
Select * from full_acs a
left join bigquery-public-data.sdoh_bea_cainc30.fips b
on b.GeoFIPS=a.geo_id;"""
query_2017_job = bigquery_client.query(query_2017)
test_base = query_2017_job.to_dataframe()

In [ ]:
df2.isna().sum()[df2.isna().sum()>0]

In [ ]:
df[df['GeoName']=='Pennington, MN']

In [ ]:
df['GeoName'].unique()

In [ ]:
df